In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights
import pytorch_lightning as pl

/home/nlp/ron.eliav/anaconda3/envs/QASemv2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class PoseDetector(pl.LightningModule):
    def __init__(self):
        super().__init__()
        weights = ResNet50_Weights.DEFAULT
        self.model = resnet50(weights=weights)
        self.model.fc = Identity()
        self.regression = nn.Linear(2048, 6)
        self.preprocess = weights.transforms()

    def forward(self, x):
        x = self.model(x)
        x = self.regression(x)
        return x

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        output = self(x)
        loss = F.mse_loss(output, y)
        self.log('train_loss', loss, prog_bar=True, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = F.mse_loss(output, y)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [3]:
img = torch.load("/home/nlp/ron.eliav/pose3d/data/resnet_dev_data/181228_000833939_Camera_0.jpg.pt")
# img = torch.load("/home/nlp/ron.eliav/pose3d/data/resnet_data/180502_025112897_Camera_0.jpg.pt")

In [4]:
img

tensor([[[ 2.1804,  2.1804,  2.1290,  ...,  2.0605,  1.8550,  1.4783],
         [ 0.9474,  1.0844,  1.5125,  ...,  2.0605,  1.5982,  1.1358],
         [-0.7479, -0.6452,  0.1254,  ...,  1.7180,  1.2214,  1.0159],
         ...,
         [ 0.2453,  0.2453,  0.2624,  ..., -1.2103, -1.2103, -1.1932],
         [ 0.2282,  0.2282,  0.2453,  ..., -1.2103, -1.2445, -1.2617],
         [ 0.2453,  0.2453,  0.2453,  ..., -1.2103, -1.2445, -1.2617]],

        [[ 2.4111,  2.4111,  2.3235,  ...,  2.0784,  1.7108,  1.4132],
         [ 1.7283,  1.7458,  1.9909,  ...,  1.9559,  1.3081,  0.9755],
         [-0.2850, -0.2150,  0.3452,  ...,  1.6758,  1.1155,  0.9230],
         ...,
         [ 1.0805,  1.0805,  1.0805,  ..., -1.0728, -1.1078, -1.0903],
         [ 1.0805,  1.0630,  1.0805,  ..., -1.1078, -1.1429, -1.1078],
         [ 1.0805,  1.0805,  1.0630,  ..., -1.1078, -1.1078, -1.1078]],

        [[ 2.5877,  2.5877,  2.4657,  ...,  1.8208,  1.5594,  1.2108],
         [ 1.9428,  1.9603,  2.2043,  ...,  1

In [5]:
model = PoseDetector.load_from_checkpoint("/home/nlp/ron.eliav/pose3d/ai-camera-pose-network/src/lightning_logs/version_9/checkpoints/epoch=108-step=3270.ckpt")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model = PoseDetector.load_from_checkpoint("/home/nlp/ron.eliav/pose3d/ai-camera-pose-net     │
│   2                                                                                              │
│                                                                                                  │
│ /home/nlp/ron.eliav/anaconda3/envs/QASemv2/lib/python3.8/site-packages/pytorch_lightning/core/mo │
│ dule.py:1531 in load_from_checkpoint                                                             │
│                                                                                                  │
│   1528 │   │   │   pretrained_model.freeze()                                                     │
│   1529 │   │   │   y_hat = pretrained_model(x)                                                   │
│   1530 │   │   """                                                                               │
│ ❱ 1531 │   │   loaded = _load_from_checkpoint(                                                   │
│   1532 │   │   │   cls,                                                                          │
│   1533 │   │   │   checkpoint_path,                                                              │
│   1534 │   │   │   map_location,                                                                 │
│                                                                                                  │
│ /home/nlp/ron.eliav/anaconda3/envs/QASemv2/lib/python3.8/site-packages/pytorch_lightning/core/sa │
│ ving.py:90 in _load_from_checkpoint                                                              │
│                                                                                                  │
│    87 │   if issubclass(cls, pl.LightningDataModule):                                            │
│    88 │   │   return _load_state(cls, checkpoint, **kwargs)                                      │
│    89 │   if issubclass(cls, pl.LightningModule):                                                │
│ ❱  90 │   │   return _load_state(cls, checkpoint, strict=strict, **kwargs)                       │
│    91 │   raise NotImplementedError(f"Unsupported {cls}")                                        │
│    92                                                                                            │
│    93                                                                                            │
│                                                                                                  │
│ /home/nlp/ron.eliav/anaconda3/envs/QASemv2/lib/python3.8/site-packages/pytorch_lightning/core/sa │
│ ving.py:149 in _load_state                                                                       │
│                                                                                                  │
│   146 │                                                                                          │
│   147 │   # load the state_dict on the model automatically                                       │
│   148 │   assert strict is not None                                                              │
│ ❱ 149 │   keys = obj.load_state_dict(checkpoint["state_dict"], strict=strict)                    │
│   150 │                                                                                          │
│   151 │   if not strict:                                                                         │
│   152 │   │   if keys.missing_keys:                                                              │
│                                                                                                  │
│ /home/nlp/ron.eliav/anaconda3/envs/QASemv2/lib/python3.8/si

In [6]:
# expand tensor dimensions to match model input
img = torch.unsqueeze(img, 0)
model(img)
# dev: 34132.3559,41088.3516,12.0351,161.386998058,87.677117148,-50.072881833
# train: 34768.9649,42018.1341,15.2489,153.73610589,85.94014665,-40.957096437


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 # expand tensor dimensions to match model input                                              │
│   2 img = torch.unsqueeze(img, 0)                                                                │
│ ❱ 3 model(img)                                                                                   │
│   4 # dev: 34132.3559,41088.3516,12.0351,161.386998058,87.677117148,-50.072881833                │
│   5 # train: 34768.9649,42018.1341,15.2489,153.73610589,85.94014665,-40.957096437                │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [14]:
torch.tensor([34132.3559,41088.3516,12.0351,161.386998058,87.677117148,-50.072881833])

tensor([ 3.4132e+04,  4.1088e+04,  1.2035e+01,  1.6139e+02,  8.7677e+01,
        -5.0073e+01])